In [6]:
import sys

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")

import pandas as pd
from tqdm.auto import tqdm

import tasks.run_helper

In [7]:
df = pd.read_json("../data/toxicity_ratings.json", lines=True)
df = df.explode(column="ratings")

ratings_df = pd.json_normalize(df.ratings)
df = pd.concat([df.reset_index(), ratings_df.reset_index()], axis=1)
df = df.drop(columns=["ratings", "index"])

df = df.loc[
    :,
    [
        "comment",
        "toxic_score",
        "personally_seen_toxic_content",
        "personally_been_target",
        "identify_as_transgender",
        "toxic_comments_problem",
        "education",
        "age_range",
        "lgbtq_status",
        "political_affilation",
        "is_parent",
        "religion_important",
    ],
]
df = df.groupby("comment").agg(list)
df["fake_index"] = 0
df = df.head(5000)
df

toxic_score  \
comment                                                               
 "Bro, can you spot me?" &amp;#x200B; I just sh...  [3, 0, 1, 4, 0]   
 "Never" as in "not often, and certainly not in...  [0, 1, 1, 4, 3]   
 "We are not talking about all white people         [2, 3, 0, 2, 0]   
 "Yeah well if we were made in his perfect imag...  [0, 2, 1, 4, 0]   
 "i'm offended" - tell them that you're offende...  [0, 1, 0, 0, 0]   
...                                                             ...   
AND SHE WILL WIN. NO OTHER WAY TO LOOK AT IT. I...  [1, 1, 0, 1, 0]   
AND THAT BLUSH. I THINK YOURE TRYING TO KILL ME     [2, 0, 2, 0, 0]   
AND THERE IS OUR POINT. STAY WOKE. STAY EDUCATE...  [0, 0, 0, 0, 1]   
AND THEY SAID WE ARNT GOING TO WAR!                 [0, 0, 3, 0, 0]   
AND THEY WERE LIKE. SUCKING FACE I DONT WANNA S...  [1, 1, 2, 2, 3]   

                                                        personally_seen_toxic_content  \
comment                                                                                 
 "Bro, can you spot me?" &amp;#x200B; I just sh...    [True, True, True, False, True]   
 "Never" as in "not often, and certainly not in...  [False, False, True, False, True]   
 "We are not talking about all white people           [True, True, False, True, True]   
 "Yeah well if we were made in his perfect imag...   [True, True, True, False, False]   
 "i'm offended" - tell them that you're offende...   [True, False, True, False, True]   
...                                                                               ...   
AND SHE WILL WIN. NO OTHER WAY TO LOOK AT IT. I...    [False, True, True, True, True]   
AND THAT BLUSH. I THINK YOURE TRYING TO KILL ME     [False, False, False, True, True]   
AND THERE IS OUR POINT. STAY WOKE. STAY EDUCATE...    [False, True, True, True, True]   
AND THEY SAID WE ARNT GOING TO WAR!                   [False, True, True, True, True]   
AND THEY WERE LIKE. SUCKING FACE I DONT WANNA S...   [True, False, False, True, True]   

                                                                 personally_been_target  \
comment                                                                                   
 "Bro, can you spot me?" &amp;#x200B; I just sh...      [True, True, True, False, True]   
 "Never" as in "not often, and certainly not in...    [False, False, True, False, True]   
 "We are not talking about all white people            [True, True, False, True, False]   
 "Yeah well if we were made in his perfect imag...   [True, False, False, False, False]   
 "i'm offended" - tell them that you're offende...   [False, False, True, False, False]   
...                                                                                 ...   
AND SHE WILL WIN. NO OTHER WAY TO LOOK AT IT. I...  [False, False, False, False, False]   
AND THAT BLUSH. I THINK YOURE TRYING TO KILL ME     [False, False, False, False, False]   
AND THERE IS OUR POINT. STAY WOKE. STAY EDUCATE...   [False, False, False, False, True]   
AND THEY SAID WE ARNT GOING TO WAR!                   [False, False, True, False, True]   
AND THEY WERE LIKE. SUCKING FACE I DONT WANNA S...   [True, False, False, False, False]   

                                                                identify_as_transgender  \
comment                                                                                   
 "Bro, can you spot me?" &amp;#x200B; I just sh...                 [No, No, No, No, No]   
 "Never" as in "not often, and certainly not in...                 [No, No, No, No, No]   
 "We are not talking about all white people                       [No, No, No, Yes, No]   
 "Yeah well if we were made in his perfect imag...                 [No, No, No, No, No]   
 "i'm offended" - tell them that you're offende...                 [No, No, No, No, No]   
...                                                                                 ...   
AND SHE WILL WIN. NO OTHER WAY TO LOOK AT IT. I...            

In [8]:
sdb_columns = [
    "personally_seen_toxic_content",
    "personally_been_target",
    "identify_as_transgender",
    "toxic_comments_problem",
    "education",
    "age_range",
    "lgbtq_status",
    "political_affilation",
    "is_parent",
    "religion_important",
]


for sdb_column in tqdm(sdb_columns):
    print("*" * 10)
    print(sdb_column)
    print(
        tasks.run_helper.results(
            df,
            discussion_id_col="fake_index",
            sdb_column=sdb_column,
            value_col="toxic_score",
            comment_key_col="comment",
        )
    )

  0%|          | 0/10 [00:00<?, ?it/s]

**********
personally_seen_toxic_content


  0%|          | 0/1 [00:00<?, ?it/s]

              0
False  0.007374
True   0.999995
**********
personally_been_target


  0%|          | 0/1 [00:00<?, ?it/s]

...

                          0
Independent        0.835793
Liberal            0.999900
Other              0.796955
Conservative       0.048567
Prefer not to say  0.788059
**********
is_parent


  0%|          | 0/1 [00:00<?, ?it/s]

                          0
Yes                0.000039
No                 1.000000
Prefer not to say  1.000000
**********
religion_important


  0%|          | 0/1 [00:00<?, ?it/s]

                           0
Not important       1.000000
Very important      0.000730
Somewhat important  0.153289
Not too important   1.000000
Prefer not to say   1.000000
